# Create Contours for DAVIS 2016 Annotations

In this notebook the contour points of DAVIS 2016 annotations are saved as numpy arrays.

## Example

For a ground truth annotation from the DAVIS 2016 dataset, the contour with the most points is extracted. Then, the number of contour points is shortened to a fixed number and the contour is saved as a numpy array. 


<img src="Images/bear_annotation.jpeg" alt="bear_annotation" width="512"/>

<img src="Images/bear_annotation_long_contour.jpeg" alt="bear_annotation_long_contour" width="512"/>

<img src="Images/bear_annotation_short_contour.jpeg" alt="bear_annotation_short_contour" width="512"/>

## Imports

In [10]:
import cv2
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import numpy as np
import os
import pandas as pd
import re
from scipy import spatial
from skimage import measure

%matplotlib inline

## Paths & Constants

In [11]:
ANNOTATIONS_480P_FOLDERS_PATH = "DAVIS_2016/DAVIS/Annotations/480p/"
CONTOURS_480P_FOLDERS_PATH = "DAVIS_2016/DAVIS/Contours/480p/"

ANNOTATIONS_1080P_FOLDERS_PATH = "DAVIS_2016/DAVIS/Annotations/1080p/"
CONTOURS_1080P_FOLDERS_PATH = "DAVIS_2016/DAVIS/Contours/1080p/"

KERNEL_SIZE = 25
CONTOUR_POINTS = 256

## Functions

In [12]:
def save_image_with_contour(image, contour, path):
    '''Saves image plotted with contour to a given path.'''
    
    fig, ax = plt.subplots(figsize=(10,7))
    
    # Plot contour
    ax.scatter(contour[:, 0], contour[:, 1], s=1)
    
    # Plot image
    #ax.imshow(image)
    ax.axis('image')
    ax.axis('off')
    ax.set_xticks([])
    ax.set_yticks([])
    
    # Save image
    plt.savefig(path, dpi=100, bbox_inches=0)    
    plt.close(fig)

In [13]:
def close_image(image, kernel_size):
    '''Returns the image that is closed with a elliptical kernel.'''
    
    kernel = cv2.getStructuringElement(cv2.MORPH_ELLIPSE,(kernel_size, kernel_size))
    closing = cv2.morphologyEx(image, cv2.MORPH_CLOSE, kernel)
    
    return closing

In [14]:
def extract_longest_contour(image, kernel_size):
    '''Returns the contour with the most points from a given image.'''
    
    # Close image
    closing = close_image(image, kernel_size)
    
    # Convert image to grayscale
    gray = cv2.cvtColor(closing, cv2.COLOR_BGR2GRAY)

    # Apply threshold to turn it into binary image
    ret, thresh = cv2.threshold(gray, 127, 255, 0)

    # Find contour
    # Change method for different number of points:
    # CHAIN_APPROX_NONE, CHAIN_APPROX_SIMPLE, CHAIN_APPROX_TC89_L1, CHAIN_APPROX_TC89_KCOS
    contours, _ = cv2.findContours(image=thresh,
                                   mode=cv2.RETR_TREE,
                                   method=cv2.CHAIN_APPROX_NONE)
    
    # Get longest contour from contours
    longest_contour = max(contours, key=len)
    
    # Remove single-dimensional entry from the shape of the array 
    longest_contour = np.squeeze(longest_contour)
    
    return longest_contour

In [15]:
def shorten_contour(contour, points):
    '''Returns a contour with a fixed number of points for a given contour.'''
    
    try:
        step = len(contour) / (len(contour) - points)
    except ZeroDivisionError as e:
        print ('len(contour) = {}, points = {}'.format(len(contour), points))
        raise ValueError from e
        
    short_contour = np.delete(contour, np.arange(0, contour.size, step), axis=0)
    
    return short_contour

In [16]:
def get_contour_from_annotation(annotation_path, kernel_size, points):
    '''Returns the image, the longest contour, and shortened contour 
       given an annotation and fixed number of points.'''
    
    image = cv2.imread(annotation_path)
    contour = extract_longest_contour(image, kernel_size)
    try:
        short_contour = shorten_contour(contour, points)
    except ValueError as e:
        print(annotation_path)
        raise ValueError from e
        
    return image, contour, short_contour

In [17]:
def create_contours_for_all_annotations(annotations_folders_path, 
                                        contours_folders_path, 
                                        kernel_size,
                                        contour_points):
    '''Creates contours for annotations and saves them as numpy arrays.'''

    # Get list of annotation folders (there is one folder for each seqeuence)
    annotations_folders_list = os.listdir(annotations_folders_path)
    annotations_folders_list.sort()
    
    # Iterate through folders
    for i, folder in enumerate(annotations_folders_list):
        
        # Debug
        if (i > 0): break
        
        print('#{}: {}'.format(i, folder))
        
        # Create folder to save contours
        contours_folder = os.path.join(contours_folders_path, folder)
        if not os.path.exists(contours_folder):
            os.makedirs(contours_folder)
        
        # Get list of annotations (there is one annotation for each frame)
        annotations = os.listdir(os.path.join(annotations_folders_path, folder))
        if '.ipynb_checkpoints' in annotations:
            annotations.remove('.ipynb_checkpoints')
        annotations.sort()
            
        # Iterate through annotations
        for j, annotation in enumerate(annotations):
            
            # Debug
            if (j > 30): break

            annotation_path = os.path.join(annotations_folders_path, folder, annotation)

            # If first mask, then extract contour_points
            if re.match(r"00000.png", annotation):
                try:
                    image, contour, short_contour = get_contour_from_annotation(
                        annotation_path, kernel_size, contour_points)
                except ValueError as e:
                    continue
            # If not, then contour_points*2
            else:
                try:
                    image, contour, short_contour = get_contour_from_annotation(
                        annotation_path, kernel_size, contour_points * 1.1)
                except ValueError as e:
                    continue

            # Save contour
            save_image_with_contour(image, short_contour, os.path.join(contours_folder, annotation[:5] + '.jpg'))
            np.save(os.path.join(contours_folder, annotation[:5]), short_contour)

## Create Contours

In [18]:
create_contours_for_all_annotations(ANNOTATIONS_480P_FOLDERS_PATH,
                                    CONTOURS_480P_FOLDERS_PATH,
                                    KERNEL_SIZE, 
                                    CONTOUR_POINTS)

#0: bear


/home/christoph/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:10: DeprecationWarning: using a non-integer array as obj in delete will result in an error in the future
  # Remove the CWD from sys.path while we load stuff.
/home/christoph/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:10: DeprecationWarning: in the future out of bounds indices will raise an error instead of being ignored by `numpy.delete`.
  # Remove the CWD from sys.path while we load stuff.
